In [1]:
from collections import defaultdict
from itertools import product

import pandas as pd
import torch

import roach
from relbench.data import TaskType
from relbench.datasets import get_dataset_names
from relbench.tasks import get_task_names, get_task

In [2]:
all_stores = roach.scan("relbench/2024-07-01")

In [3]:
len(all_stores)

369

In [4]:
all_stores[-1]["__roach__"]

{'project': 'relbench/2024-07-01',
 'timestamp': 1719951248879018973,
 'caller_file': 'gnn_node.py',
 'done': True}

In [5]:
def wrap(name):
    return r"\texttt{" + name + r"}"

In [6]:
txt = {
    "val": "Val",
    "test": "Test",
}

# classification

In [18]:
metric = "roc_auc"
higher_is_better = True

In [19]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value not in [
            TaskType.BINARY_CLASSIFICATION.value,
            TaskType.MULTICLASS_CLASSIFICATION.value,
        ]:
            continue
        for script in [
            "gnn_node",
            "lightgbm_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                vals = []
                for seed in range(5):
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task
                            and store["args"]["seed"] == seed,
                            all_stores,
                        )
                    )
                    try:
                        store = stores[-1]
                        val = store[split][metric]
                        vals.append(val)
                    except IndexError:
                        # val = float("nan")
                        # vals.append(val)
                        pass
                val = torch.tensor(vals)
                mean = val.mean().item()
                std = val.std().item()
                record = {
                    "dataset": dataset,
                    "task": task,
                    "script": script,
                    "split": split,
                    "mean": mean,
                    "std": std,
                }
                table_data.append(record)

In [20]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    tex_val = (
        r"$"
        + opt_bm_open
        + f"{mean * 100:.2f}"
        + opt_bm_close
        + r"_{\pm "
        + f"{std * 100:.2f}"
        + r"}$"
    )

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-churn}      Val   $\bm{70.47}_{\pm 0.02}$   
                                             Test  $\bm{70.40}_{\pm 0.08}$   
                    \texttt{item-churn}      Val   $\bm{82.41}_{\pm 0.03}$   
                                             Test  $\bm{82.79}_{\pm 0.01}$   
\texttt{rel-avito}  \texttt{user-visits}     Val   $\bm{69.66}_{\pm 0.09}$   
                                             Test  $\bm{66.06}_{\pm 0.16}$   
                    \texttt{user-clicks}     Val   $\bm{65.48}_{\pm 0.89}$   
                                             Test  $\bm{67.26}_{\pm 1.18}$   
\texttt{rel-event}  \texttt{user-repeat}     Val   $\bm{68.72}_{\pm 0.94}$   
                                             Test  $\bm{77.55}_{\pm 0.75}$   
                    \texttt{user-ignore}     Val   $\bm{91.06}_{\pm 0.56}$   
                                             Test  $\bm{79.34}_{\pm 1.03}$   
\texttt{rel-f1}     \texttt{driver-dnf}      Val   $\bm{72.82}_{\pm 0.58}$   
                                             Test  $\bm{74.13}_{\pm 1.36}$   
                    \texttt{driver-top3}     Val   $\bm{73.60}_{\pm 4.85}$   
                                             Test  $\bm{80.06}_{\pm 0.50}$   
\texttt{rel-hm}     \texttt{user-churn}      Val   $\bm{70.44}_{\pm 0.05}$   
                                             Test  $\bm{69.72}_{\pm 0.15}$   
\texttt{rel-stack}  \texttt{user-engagement} Val   $\bm{90.17}_{\pm 0.02}$   
                                             Test  $\bm{90.57}_{\pm 0.02}$   
                    \texttt{user-badge}      Val   $\bm{89.81}_{\pm 0.03}$   
                                             Test  $\bm{88.80}_{\pm 0.02}$   
\texttt{rel-trial}  \texttt{study-outcome}   Val        $64.91_{\pm 0.18}$   
                                             Test       $67.45_{\pm 0.58}$   

                                                         lightgbm_baseline  
\texttt{rel-amazon} \texttt{user-churn}      Val        $52.21_{\pm 0.04}$  
                                             Test       $52.25_{\pm 0.08}$  
                    \texttt{item-churn}      Val        $62.31_{\pm 0.16}$  
                                             Test       $62.75_{\pm 0.21}$  
\texttt{rel-avito}  \texttt{user-visits}     Val        $53.23_{\pm 0.07}$  
                                             Test       $53.03_{\pm 0.26}$  
                    \texttt{user-clicks}     Val        $55.90_{\pm 0.26}$  
                                             Test       $54.08_{\pm 0.56}$  
\texttt{rel-event}  \texttt{user-repeat}     Val   $\bm{68.19}_{\pm 0.32}$  
                                             Test       $67.69_{\pm 2.85}$  
                    \texttt{user-ignore}     Val        $87.88_{\pm 0.45}$  
                                             Test  $\bm{79.46}_{\pm 0.71}$  
\texttt{rel-f1}     \texttt{driver-dnf}      Val        $68.91_{\pm 0.52}$  
                                             Test  $\bm{69.32}_{\pm 3.78}$  
                    \texttt{driver-top3}     Val   $\bm{72.31}_{\pm 2.30}$  
                                             Test  $\bm{73.50}_{\pm 7.56}$  
\texttt{rel-hm}     \texttt{user-churn}      Val        $56.10_{\pm 0.10}$  
                                             Test       $55.22_{\pm 0.12}$  
\texttt{rel-stack}  \texttt{user-engagement} Val        $64.31_{\pm 0.18}$  
                                             Test       $63.69_{\pm 0.20}$  
                    \texttt{user-badge}      Val        $65.19_{\pm 0.21}$  
                                             Test       $63.25_{\pm 0.29}$  
\texttt{rel-trial}  \texttt{study-outcome}   Val   $\bm{67.78}_{\pm 0.36}$  
                                             Test  $\bm{70.16}_{\pm 1.52}$

In [21]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-5} \cmidrule{2-5}", r"\cmidrule{1-5}")
print(tex)

\begin{tabular}{lllll}
\toprule
 &  &  & gnn_node & lightgbm_baseline \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-churn}} & Val & $\bm{70.47}_{\pm 0.02}$ & $52.21_{\pm 0.04}$ \\
 &  & Test & $\bm{70.40}_{\pm 0.08}$ & $52.25_{\pm 0.08}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{item-churn}} & Val & $\bm{82.41}_{\pm 0.03}$ & $62.31_{\pm 0.16}$ \\
 &  & Test & $\bm{82.79}_{\pm 0.01}$ & $62.75_{\pm 0.21}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{user-visits}} & Val & $\bm{69.66}_{\pm 0.09}$ & $53.23_{\pm 0.07}$ \\
 &  & Test & $\bm{66.06}_{\pm 0.16}$ & $53.03_{\pm 0.26}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{user-clicks}} & Val & $\bm{65.48}_{\pm 0.89}$ & $55.90_{\pm 0.26}$ \\
 &  & Test & $\bm{67.26}_{\pm 1.18}$ & $54.08_{\pm 0.56}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-event}} & \multirow[c]{2}{*}{\texttt{user-repeat}} & Val & $\bm{68.72}_{\pm 0.94}$ & $\bm{68.19}_{\pm 0.32}$

# regression

In [22]:
metric = "mae"
higher_is_better = False

In [23]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.REGRESSION.value:
            continue
        for script in [
            "gnn_node",
            "lightgbm_baseline",
            "node_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "node_baseline":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_zero",
                        "global_mean",
                        "global_median",
                        "entity_mean",
                        "entity_median",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

In [24]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_node",
        "lightgbm_baseline",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean:.3f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std:.3f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean:.3f}" + opt_bm_close + r"$"

    rec["tex"] = tex_val

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val

tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-ltv}        Val   $\bm{12.135}_{\pm 0.014}$   
                                             Test  $\bm{14.333}_{\pm 0.043}$   
                    \texttt{item-ltv}        Val   $\bm{45.237}_{\pm 0.138}$   
                                             Test  $\bm{50.330}_{\pm 0.193}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val    $\bm{0.037}_{\pm 0.001}$   
                                             Test   $\bm{0.041}_{\pm 0.001}$   
\texttt{rel-event}  \texttt{user-attendance} Val    $\bm{0.261}_{\pm 0.002}$   
                                             Test   $\bm{0.265}_{\pm 0.006}$   
\texttt{rel-f1}     \texttt{driver-position} Val    $\bm{3.181}_{\pm 0.007}$   
                                             Test   $\bm{4.096}_{\pm 0.165}$   
\texttt{rel-hm}     \texttt{item-sales}      Val    $\bm{0.065}_{\pm 0.000}$   
                                             Test   $\bm{0.056}_{\pm 0.000}$   
\texttt{rel-stack}  \texttt{post-votes}      Val    $\bm{0.059}_{\pm 0.000}$   
                                             Test   $\bm{0.065}_{\pm 0.000}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val   $\bm{46.263}_{\pm 0.238}$   
                                             Test  $\bm{44.352}_{\pm 0.402}$   
                    \texttt{site-success}    Val    $\bm{0.387}_{\pm 0.000}$   
                                             Test   $\bm{0.383}_{\pm 0.032}$   

                                                           lightgbm_baseline  \
\texttt{rel-amazon} \texttt{user-ltv}        Val        $14.141_{\pm 0.000}$   
                                             Test       $16.783_{\pm 0.000}$   
                    \texttt{item-ltv}        Val        $55.800_{\pm 0.052}$   
                                             Test       $60.639_{\pm 0.041}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val    $\bm{0.037}_{\pm 0.000}$   
                                             Test   $\bm{0.041}_{\pm 0.000}$   
\texttt{rel-event}  \texttt{user-attendance} Val         $0.264_{\pm 0.000}$   
                                             Test   $\bm{0.266}_{\pm 0.000}$   
\texttt{rel-f1}     \texttt{driver-position} Val         $3.477_{\pm 0.022}$   
                                             Test   $\bm{4.129}_{\pm 0.073}$   
\texttt{rel-hm}     \texttt{item-sales}      Val         $0.086_{\pm 0.000}$   
                                             Test        $0.076_{\pm 0.000}$   
\texttt{rel-stack}  \texttt{post-votes}      Val         $0.062_{\pm 0.000}$   
                                             Test        $0.068_{\pm 0.000}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val   $\bm{44.014}_{\pm 2.038}$   
                                             Test  $\bm{42.657}_{\pm 1.851}$   
                    \texttt{site-success}    Val         $0.415_{\pm 0.001}$   
                                             Test        $0.424_{\pm 0.001}$   

                                                  global_zero global_mean  \
\texttt{rel-amazon} \texttt{user-ltv}        Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
                    \texttt{item-ltv}        Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-event}  \texttt{user-attendance} Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-f1}     \texttt{driver-position} Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-hm}     \texttt{item-sales}      Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-stack}  \text

In [25]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-10} \cmidrule{2-10}", r"\cmidrule{1-10}")
print(tex)

\begin{tabular}{llllllllll}
\toprule
 &  &  & gnn_node & lightgbm_baseline & global_zero & global_mean & global_median & entity_mean & entity_median \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-ltv}} & Val & $\bm{12.135}_{\pm 0.014}$ & $14.141_{\pm 0.000}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{14.333}_{\pm 0.043}$ & $16.783_{\pm 0.000}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{2-10}
 & \multirow[c]{2}{*}{\texttt{item-ltv}} & Val & $\bm{45.237}_{\pm 0.138}$ & $55.800_{\pm 0.052}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{50.330}_{\pm 0.193}$ & $60.639_{\pm 0.041}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{1-10}
\multirow[c]{2}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{ads-clicks}} & Val & $\bm{0.037}_{\pm 0.001}$ & $\bm{0.037}_{\pm 0.000}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{na

# link prediction

In [30]:
metric = "link_prediction_map"
higher_is_better = True

In [31]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.LINK_PREDICTION.value:
            continue
        for script in [
            "gnn_link",
            "idgnn_link",
            "lightgbm_link_baseline",
            "link_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "link_baseline":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_popularity",
                        "past_visit",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"] == f"{script}.py"
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

/tmp/user/23092/ipykernel_4018174/2770183924.py:66: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std = val.std().item()


In [32]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_link",
        "idgnn_link",
        "lightgbm_link_baseline",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean * 100:.2f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std * 100:.2f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean * 100:.2f}" + opt_bm_close + r"$"

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val   $\bm{1.48}_{\pm 0.01}$   
                                                   Test  $\bm{0.69}_{\pm 0.04}$   
                    \texttt{user-item-rate}        Val   $\bm{1.48}_{\pm 0.04}$   
                                                   Test  $\bm{0.89}_{\pm 0.10}$   
                    \texttt{user-item-review}      Val   $\bm{1.07}_{\pm 0.07}$   
                                                   Test  $\bm{0.40}_{\pm 0.03}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val        $0.82_{\pm 0.04}$   
                                                   Test       $0.78_{\pm 0.02}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val        $0.60_{\pm 0.05}$   
                                                   Test       $0.21_{\pm 0.03}$   
                    \texttt{post-post-related}     Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val        $3.38_{\pm 0.14}$   
                                                   Test       $3.16_{\pm 0.57}$   
                    \texttt{site-sponsor-run}      Val   $\bm{15.83}_{\pm nan}$   
                                                   Test  $\bm{12.26}_{\pm nan}$   

                                                                      idgnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val         $0.13_{\pm 0.00}$   
                                                   Test        $0.10_{\pm 0.00}$   
                    \texttt{user-item-rate}        Val         $0.15_{\pm 0.00}$   
                                                   Test        $0.12_{\pm 0.00}$   
                    \texttt{user-item-review}      Val         $0.11_{\pm 0.00}$   
                                                   Test        $0.09_{\pm 0.00}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val    $\bm{5.43}_{\pm 0.02}$   
                                                   Test   $\bm{3.66}_{\pm 0.02}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val    $\bm{2.61}_{\pm 0.00}$   
                                                   Test   $\bm{2.80}_{\pm 0.01}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val   $\bm{15.25}_{\pm 0.20}$   
                                                   Test  $\bm{12.78}_{\pm 0.20}$   
                    \texttt{post-post-related}     Val    $\bm{7.94}_{\pm 0.17}$   
                                                   Test  $\bm{10.60}_{\pm 0.05}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val   $\bm{11.49}_{\pm 0.02}$   
                                                   Test  $\bm{11.48}_{\pm 0.17}$   
                    \texttt{site-sponsor-run}      Val   $\bm{17.50}_{\pm 0.02}$   
                                                   Test  $\bm{19.07}_{\pm 0.08}$   

                                                        lightgbm_link_baseline  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-rate}        Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-review}      Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan

In [33]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-7} \cmidrule{2-7}", r"\cmidrule{1-7}")
print(tex)

\begin{tabular}{llllllll}
\toprule
 &  &  & gnn_link & idgnn_link & lightgbm_link_baseline & global_popularity & past_visit \\
\midrule
\multirow[c]{6}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-item-purchase}} & Val & $\bm{1.48}_{\pm 0.01}$ & $0.13_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.31$ & $0.08$ \\
 &  & Test & $\bm{0.69}_{\pm 0.04}$ & $0.10_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.24$ & $0.06$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-rate}} & Val & $\bm{1.48}_{\pm 0.04}$ & $0.15_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.16$ & $0.09$ \\
 &  & Test & $\bm{0.89}_{\pm 0.10}$ & $0.12_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.15$ & $0.07$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-review}} & Val & $\bm{1.07}_{\pm 0.07}$ & $0.11_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.18$ & $0.04$ \\
 &  & Test & $\bm{0.40}_{\pm 0.03}$ & $0.09_{\pm 0.00}$ & $\bm{nan}_{\pm nan}$ & $0.11$ & $0.04$ \\
\cmidrule{1-8} \cmidrule{2-8}
\multirow[c]{2}{*}{\texttt{rel-av